In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
#geopy installation

!pip install geopy
import geopy 

from geopy.geocoders import GoogleV3
import time
from pprint import pprint
geolocator = GoogleV3(api_key="")

In [3]:
#Installing shortuuid to make dorms identifiable (Some Dorm Names Suck!)

!pip install shortuuid

import shortuuid    
print(shortuuid.ShortUUID().random(length=6)) #Example Code 

3v9c5z


In [4]:
url = "https://www.kollegierneskontor.dk/default.aspx?func=kkikportal.kollegiumlist&mid=40&topmenuid=34&lang=GB"
results = requests.get(url)

In [5]:
soup = BeautifulSoup(results.text, "html.parser")


In [6]:
#Longitude and latitude

def coordinates(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocator.geocode(address)
             return {"latitude":location.latitude, "longitude":location.longitude}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":"NaN", "longitude":"NaN"}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location

#print(f"{latitude}, {longitude}")
# print all returned data
#pprint(location)

print(coordinates("Dalslandsgade 8, Copenhagen S"))

Couldn'd find coordinates of Dalslandsgade 8, Copenhagen S
{'latitude': 'NaN', 'longitude': 'NaN'}


In [7]:

kkik = []

kollegium = soup.find_all('div', class_='kollegium row')


for container in kollegium:
    print("Processing Dorm")
    dorm_name = container.h3.a.text
    dorm_address = container.div.find('div', class_="address").text.strip().replace("\r\n               ", "").replace("\xa0", "")
    dorm_picture_link = container.find('img', class_="kollegiumlistimage img")['src']
    print(dorm_picture_link)

    body = container.find('div', class_="body col-xs-12")
    rows = body.find_all("div", class_= "row")
    rows.pop(0)
    rows.pop(0)
    rows.pop(0)

    rooms_export = []

    for room in rows:
        room_details = room.find_all("div")

        roomtype = room_details[0].get_text().strip()
        size = room_details[1].get_text().strip()
        price = room_details[3].get_text().strip()

        rooms_export.append({
            "roomType":roomtype,
            "size":size,
            "price":price
        })

    kkik.append({
        "name":dorm_name,
        "address":dorm_address,
        "dormPictureLink":"https://www.kollegierneskontor.dk/"+dorm_picture_link
        "coordinates":coordinates(dorm_address),
        "rooms":rooms_export,
        "UUID":shortuuid.ShortUUID().random(length=6)
    })








Processing Dorm
getadminimage.aspx?imagename=9ee7415c-f181-46fd-a911-d65faa4f4956.JPG&params=w150&w100
Couldn'd find coordinates of Morbærhaven 0 , 2620 Albertslund
Processing Dorm
getadminimage.aspx?imagename=8be6df9c-fbb2-4dc6-aff1-4fd1316afd58.jpg&params=w150&w100
Couldn'd find coordinates of Amagerbrogade 268 , 2300 København S
Processing Dorm
getadminimage.aspx?imagename=97f987a2-20ab-47ae-bc38-8c27fb6ca55b.JPG&params=w150&w100
Couldn'd find coordinates of Dahlerupsgade 1 , 1603 København V
Processing Dorm
getadminimage.aspx?imagename=e3e83082-f8c5-47e9-a3b4-fe6dcfbd5012.JPG&params=w150&w100
Couldn'd find coordinates of Vognporten 14 , 2620 Albertslund
Processing Dorm
getadminimage.aspx?imagename=39b532b1-f6c7-4b23-af63-2108d52dcf40.JPG&params=w150&w100
Couldn'd find coordinates of Emdrupvej 54 A, 2400 København NV
Processing Dorm
getadminimage.aspx?imagename=dffc7704-7e89-4c9c-a610-6e0077270b3a.JPG&params=w150&w100
Couldn'd find coordinates of Sylows Allé 13 , 2000 Frederiksberg


TypeError: 'NoneType' object is not subscriptable

In [28]:
import json

with open('json_kkik.json', 'w') as f:
    json.dump(kkik, f)


3Y9pHV
